## Scalability Testing

This notebook is for testing purposes.

In [1]:
from config import DATA_PATH, Color
from helpers import clean
from helpers.coordinates import ENUPose, GRAPose
from planner import Plan
from planner.plans.auto import AutoPlan
from simulator import NoVisualizer, Simulator
from simulator.sim import SimVehicle

clean()

## Simulation Positions

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs_colors=[Color.RED,Color.ORANGE,Color.GREEN,Color.BLUE]
n_uavs_per_gcs = 60
side_len = 10
altitude = 5

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs_colors))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

In [3]:
enu_homes = enu_origin.to_abs_all(base_homes)
gra_homes = gra_origin.to_abs_all(base_homes)
enu_wptrajs = [enu_home.to_abs_all(base_path) 
            for enu_home, base_path in zip(enu_homes, base_paths)]
gra_wptrajs = [gra_home.to_abs_all(base_path) 
            for gra_home, base_path in zip(gra_homes, base_paths)]

## Create Vehicles

In [4]:
sysids = range(1, len(base_homes)+1)
colors: list[Color]= []
for color in gcs_colors:
    colors.extend([color]*n_uavs_per_gcs)

vehs:list[SimVehicle] = []
for sysid, enu_home, gra_wptraj, enu_wptraj, color in zip(
        sysids, enu_homes, gra_wptrajs, enu_wptrajs, colors):
    mission_path = DATA_PATH / f"mission_{sysid}.waypoints"
    plan = AutoPlan(
        name="simple_auto_plan",
        mission_path=str(mission_path),
    )
    plan.save_basic_mission(
        sysid=sysid,
        gra_wps=GRAPose.unpose_all(gra_wptraj),
    )

    veh = SimVehicle(
        sysid=sysid,
        gcs_name=f'{color.name}_{color.emoji}',
        plan=plan,
        color=color,
        home=enu_home,
        waypoints=ENUPose.unpose_all(enu_wptraj),
    )
    vehs.append(veh)
    

## Simulator

In [5]:
simulator = Simulator(
	visualizer=NoVisualizer(gra_origin),
	terminals=['gcs'],
	verbose=1,
)
for veh in vehs:
    simulator.add_vehicle(veh)

## Run

In [ ]:
orac = simulator.launch()
orac.run()

12:55:28 - Oracle ⚪ - INFO - 🙈 Running without visualization.
12:55:28 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 199216)
12:55:28 - Oracle ⚪ - INFO - 🚀 GCS ORANGE_🟧 launched (PID 199217)


12:55:28 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 199218)
12:55:28 - Oracle ⚪ - INFO - 🚀 GCS BLUE_🟦 launched (PID 199219)
12:55:28 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 240 vehicles and 4 GCSs
12:57:39 - Oracle ⚪ - INFO - UAV 138 completed mission and exited
12:57:39 - Oracle ⚪ - INFO - UAV 122 completed mission and exited
12:57:40 - Oracle ⚪ - INFO - UAV 212 completed mission and exited
12:57:40 - Oracle ⚪ - INFO - UAV 206 completed mission and exited
12:57:40 - Oracle ⚪ - INFO - UAV 69 completed mission and exited
12:57:40 - Oracle ⚪ - INFO - UAV 146 completed mission and exited
12:57:40 - Oracle ⚪ - INFO - UAV 88 completed mission and exited
12:57:40 - Oracle ⚪ - INFO - UAV 28 completed mission and exited
12:57:40 - Oracle ⚪ - INFO - UAV 19 completed mission and exited
12:57:41 - Oracle ⚪ - INFO - UAV 129 completed mission and exited
12:57:41 - Oracle ⚪ - INFO - UAV 151 completed mission and exited
12:57:41 - Oracle ⚪ - INFO - UAV 26 completed mission and exited
12:57:41